# Max Entropy学习总结

## 模型推导

最大熵模型从最基础的最大熵思想，去推出表达式。

**最大熵**是选择最优模型的一个准则，即，首先要**满足已有的事实（约束条件）**，然后在没有更多信息的情况下，那些不确定的部分都是“等可能的”。“等可能”本身不容易操作，熵是一个可以优化的数值目标。

这里最大熵最大化的，是**条件熵**$H(Y|X)$。

$$H(Y|X)=-\sum_{x,y}\hat{P}(x)P(y|x)logP(y|x)$$

具体的关于信息熵的文章，可以看[colah的这篇博客](https://colah.github.io/posts/2015-09-Visual-Information/)，里面对信息熵/互信息/条件熵的物理意义做很好的解释。

好的，我们的最大熵有了目标函数。刚才说了，我们还必须要保证满足已有的事实，这一点如何用数学公式去描述呢？

首先引入**特征函数**：

$$f(x,y)=\left\{\begin{matrix}
1, \;\;x与y满足某一事实\\ 
0，\;\;否则
\end{matrix}\right.$$

下面注意到两个期望：
- 特征函数$f(x,y)$关于经验分布$\hat P(x,y)$的期望：$E_{\hat P}(f)=\sum_{x,y}\hat P(x,y)f(x,y)$
- 特征函数$f(x,y)$关于模型$P(Y|X)$与经验分布$\hat P(x)$的期望：$E_{P}(f)=\sum_{x,y}\hat P(x)P(y|x)f(x,y)$

我们希望，模型在训练完以后，**能够获取到训练数据中的信息**。这个想法，用上面的两个期望表达，就是：

$$E_{\hat P}(f)=E_{P}(f)$$

给定了目标函数和约束条件，我们通过拉格朗日对偶法，解得模型的**更一般的形式**，（具体的求解过程省略，这里主要是展现模型思想）：

$$P_w(y|x)=\frac{1}{Z_w(x)}exp\bigl(\begin{smallmatrix}
\sum_{i=1}^{n} w_i\cdot f_i(x,y)
\end{smallmatrix}\bigr)$$

其中，$Z_w(x)$是归一化因子，$Z_w(x)=\sum_yexp\bigl(\begin{smallmatrix}\sum_{i=1}^{n} w_i\cdot f_i(x,y)
\end{smallmatrix}\bigr)$。$w \in R^n$是权值向量，$f_i(x,y)$是特征函数。

这个形式和无向图模型几乎一毛一样～

## 模型的训练

模型的训练可以直接使用极大似然法去估计参数。